In [ ]:
class Utilss:
    
    # Function to apply natural logarithm and diff to data
    @staticmethod
    def transform_diflog(x, offset = 0):
        ndiff = arima.nsdiffs(x)
        return x.transform(np.log).diff() + offset

    # Function to create embedding function
    @staticmethod
    def embed(x, m):
        x = x[~pd.isna(np.array(x))]
        emb = x.copy()
        emb.columns = ["lag_0"]
        for i in range(1, m + 1):
            emb[f"lag_{i}"] = x.shift(-i).to_numpy()
        emb = emb.dropna().reset_index(drop = True)
        emb = emb.iloc[:, ::-1]
        return emb

    # function to create train and test partition
    @staticmethod
    def train_test(embed, data, horizon):
        y_train = embed.iloc[:, :1]
        y_test = data.tail(horizon)

        X_train = embed.iloc[:,1:]
        X_test = embed.tail(1).shift(1, axis = 1).iloc[:,1:]

        return X_train, X_test, y_train, y_test

    # runction to fit data iteratively
    @staticmethod
    def forecasting(mod, X_train, y_train, **model_hyperparam):
        forecast_svr = []
        for i in range(horizon):
            model = mod(**model_hyperparam\
                            ).fit(X_train, y_train.to_numpy().reshape(-1))
            predict = model.predict(X_test)
            forecast_svr.extend(list(predict))

            y_train = y_train.iloc[1:]
            X_train = X_train.iloc[:-1]

        forecast_svr = np.array(forecast_svr)
        return forecast_svr

    # function to transform the data back from nl and diff
    @staticmethod
    def backtransform(forecast, data, offset):
        exp_term = np.exp(np.cumsum(forecast-offset))

        last_observation = data.tail(1).to_numpy()

        return last_observation * exp_term